# CityBikes

Send a request to CityBikes for the city of your choice. 

In [124]:
import requests
import json
import pandas as pd

#Richmond, Virginia was selected as the target city to perform the analysis and data modelling

#Retrieve the whole list of CityBikes Networks
Networks_URL='http://api.citybik.es/v2/networks'
networks_response=requests.get(Networks_URL)
# running networks_response.json() reveals 'networks' is the json key

#Format json as a DataFrame
json_content = networks_response.json()
Networks = pd.DataFrame(json_content['networks'])
#Networks = pd.json_normalize(Networks['location'])
Networks.head()

,company,href,id,location,name,source,gbfs_href,license,ebikes
0,[ЗАО «СитиБайк»],/v2/networks/velobike-moscow,velobike-moscow,"{'city': 'Moscow', 'country': 'RU', 'latitude'...",Velobike,NaN,NaN,NaN,NaN
1,[Urban Infrastructure Partner],/v2/networks/baerum-bysykkel,baerum-bysykkel,"{'city': 'Bærum', 'country': 'NO', 'latitude':...",Bysykkel,NaN,NaN,NaN,NaN
2,[Comunicare S.r.l.],/v2/networks/bicincitta-siena,bicincitta-siena,"{'city': 'Siena', 'country': 'IT', 'latitude':...",Bicincittà,https://www.bicincitta.com/frmLeStazioni.aspx?...,NaN,NaN,NaN
3,[Cyclopolis Systems],/v2/networks/cyclopolis-maroussi,cyclopolis-maroussi,"{'city': 'Maroussi', 'country': 'GR', 'latitud...",Cyclopolis,NaN,NaN,NaN,NaN
4,[Cyclopolis Systems],/v2/networks/cyclopolis-nafplio,cyclopolis-nafplio,"{'city': 'Nafplio', 'country': 'GR', 'latitude...",Cyclopolis,NaN,NaN,NaN,NaN


In [131]:
#Find Richmond in Networks to get its href
location_df = pd.json_normalize(Networks['location'])
index=location_df[location_df['city'].str.contains('Richmond')].index
href=Networks['href'].iloc[index[0]]

#retrieve the list of bike stations in Richmond as json
URL=f'http://api.citybik.es{href}'
response=requests.get(URL)
stations_json=response.json()

Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [137]:
#Format response json as a DataFrame showing the values inside the key 'stations'
rawdata_stations = pd.json_normalize(stations_json['network']['stations'])
rawdata_stations.head()

,empty_slots,free_bikes,id,latitude,longitude,name,timestamp,extra.address,extra.description,extra.last_update,extra.online,extra.photo,extra.uid
0,13,3,6646153247a8d51a860523106fa43b2f,37.548877,-77.442697,Abner Clay Park,2023-10-21T04:51:46.281000Z,216 W. Clay Street,"NE corner, on Clay at Brook",1697853677848,False,https://api.rvabikes.com/cxf/fm/files/00000000...,160c8c5f-2516-4131-a41b-a587294d658d
1,11,1,a3067db1c01014a5ce5b24a2272cb9ec,37.568589,-77.441463,Battery Park,2023-10-21T04:51:46.283000Z,2719 Dupont Circle,Battery Park,1697641571223,False,https://api.rvabikes.com/cxf/fm/files/00000000...,22df129a-8693-4138-b582-cb0f1e24c2c7
2,15,1,a8bf6647cec7c7dc924323ea8443956d,37.516124,-77.444372,Blackwell,2023-10-21T04:51:46.283000Z,"E 16th St & Everett St, Richmond, VA 23224",Blackwell Pool,1697640519138,False,https://api.rvabikes.com/cxf/fm/files/00000000...,e432e94e-bccf-4684-b62d-c67c07c34d57
3,18,1,3932aa7e90f065b40a3e33f72926cdc4,37.551472,-77.451937,Broad & Harrison,2023-10-21T04:51:46.293000Z,1111 W. Broad Street,Mid block,1697749148586,False,https://api.rvabikes.com/cxf/fm/files/00000000...,f606ba0f-a9f7-4c83-872a-ca28a060aa1d
4,23,0,47bc15a153b51022b6c6fbd54464d38b,37.553716,-77.455681,Broad & Lombardy,2023-10-21T04:51:46.293000Z,1371 W. Broad Street,Mid block,1696419415826,False,https://api.rvabikes.com/cxf/fm/files/00000000...,5b67664f-17b6-4abb-bf6c-85e973479794


Put your parsed results into a DataFrame.

In [152]:
#create stations DataFrame 
stations = rawdata_stations[['empty_slots','free_bikes','latitude','longitude']]
stations.head()

,empty_slots,free_bikes,latitude,longitude
0,13,3,37.548877,-77.442697
1,11,1,37.568589,-77.441463
2,15,1,37.516124,-77.444372
3,18,1,37.551472,-77.451937
4,23,0,37.553716,-77.455681


In [145]:
#save stations dataframe to json file
stations.to_json('stations.json')